# Reconocimiento de patrones: Clasificación
### Ramón Soto C. [(rsotoc@moviquest.com)](mailto:rsotoc@moviquest.com/)
![ ](images/blank.png)
![agents](images/binary_data_under_a_magnifying.jpg)
[ver en nbviewer](http://nbviewer.ipython.org/github/rsotoc/pattern-recognition/blob/master/Clasificación%20I.ipynb) 

## Cálculo de la calidad de clasificación


## La *Exactitud*

La forma más común de evaluar la calidad de un sistema de clasificación automática es a través de su *exactitud* sobre datos conocidos, esto es, la tasa de aciertos $(N_c)$ contra el total de intentos $(N_T)$:

$$AC = \frac{N_c}{N_T}$$

Calculemos la exactitud para un caso simple:

In [1]:
# Inicializar el ambiente
import numpy as np
import pandas as pd
import math
import sys

from scipy.spatial import distance
from sklearn import cluster
from matplotlib import pyplot as plt

%matplotlib inline
np.set_printoptions(precision=2, suppress=True) # Cortar la impresión de decimales a 1

Puesto que necesitamos datos previmente clasificados para entrenar el clasificador, utilizamos k-medias para asignar valores de clase a cada dato:

In [2]:
# Leer los datos de archivo, separar training y test y calcular "prototipos de clase"
df = pd.read_csv("Data sets/datosProm.csv", names = ['A', 'B'])

# Clasificar con k-means
num_clusters = 3
k_means = cluster.KMeans(n_clusters=num_clusters, init='random')
k_means.fit(df)
df["Class"] = k_means.labels_

# Desplegar los datos clasificados
display(df)

,A,B,Class
0,70.28,42.125,2
1,0.00,56.750,1
2,79.00,2.500,2
3,75.64,11.667,2
4,82.00,58.800,0
5,86.00,77.265,0
6,80.00,85.500,0
7,68.20,62.440,0
8,72.00,88.000,0
9,74.00,80.604,0


A continuación, separamos los datos en datos de entrenamiento y datos de prueba.

In [3]:
from sklearn.model_selection import train_test_split

# Seleccionar y desplegar datos de entrenamiento/prueba
df_train, df_test = train_test_split(df, test_size=0.33, random_state=3)
print("Datos de entrenamiento:\n{}\n\nDatos de Prueba:\n{}".format(df_train, df_test))

Datos de entrenamiento:
         A       B  Class
17   86.00  89.583      0
27   77.00  84.375      0
7    68.20  62.440      0
12   79.80  71.200      0
4    82.00  58.800      0
23    0.00  75.919      1
6    80.00  85.500      0
20  100.00  95.063      0
9    74.00  80.604      0
11   77.69  13.167      2
29   92.60  66.875      0
19    0.00  90.625      1
21   87.00  62.556      0
0    70.28  42.125      2
8    72.00  88.000      0
28   70.80  35.000      2
3    75.64  11.667      2
25   73.46  83.531      0
24    0.00  76.875      1
10   91.72   0.000      2

Datos de Prueba:
       A       B  Class
15   0.0  59.933      1
5   86.0  77.265      0
22  79.0  82.458      0
26  92.0  87.563      0
18   0.0  76.250      1
14   0.0  80.854      1
13  87.1  75.391      0
2   79.0   2.500      2
16  96.0   5.000      2
1    0.0  56.750      1


Utilizamos un clasificador de k-vecinos próximos para clasificar los datos de prueba:

In [4]:
from sklearn.neighbors import KNeighborsClassifier

# Crear y entrenar un artefacto KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
train_output = df_train[["Class"]].values.ravel()
neigh.fit(df_train[["A", "B"]], train_output)

# Clasificar los datos de prueba e imprimir los resultados
test_expected_output = df_test[["Class"]].values.ravel()
print("Salida esperada:", test_expected_output)
test_automatic_output = neigh.predict(df_test[["A", "B"]])
print("\nSalida obtenida:", test_automatic_output)

# Contabilizar casos correctos
correct = 0
for i in range(len(test_expected_output)):
    if test_automatic_output[i] == test_expected_output[i]:
        correct += 1

# Calcular la exactitud
accuracy = correct / len(test_expected_output)
print("\nExactitud:", accuracy)
print("\nExactitud porcentual:", accuracy * 100)
print("\nTasa de error:", (1 - accuracy) * 100)

Salida esperada: [1 0 0 0 1 1 0 2 2 1]

Salida obtenida: [1 0 0 0 1 1 0 2 2 1]

Exactitud: 1.0

Exactitud porcentual: 100.0

Tasa de error: 0.0


Probamos ahora con los datos del "Pima Indians Diabetes Dataset" (utilizamos semilla fija para reproducibilidad).

In [7]:
from sklearn import preprocessing

df_pid = pd.read_csv("Data sets/Pima Indian Data Set/pima-indians-diabetes.data", 
                 names = ['emb', 'gl2h', 'pad', 'ept', 'is2h', 'imc', 'fpd', 'edad', 'class'])
df_pid.loc[df_pid['pad'] == 0,'pad'] = np.nan
df_pid.loc[df_pid['ept'] == 0,'ept'] = np.nan
df_pid.loc[df_pid['is2h'] == 0,'is2h'] = np.nan
df_pid.loc[df_pid['imc'] == 0,'imc'] = np.nan
df_pid = df_pid.dropna()

df_train2, df_test2 = train_test_split(df_pid, test_size=0.33, random_state=0)

# Crear y entrenar un artefacto KNeighborsClassifier
neigh2 = KNeighborsClassifier(n_neighbors=5)
train_output2 = df_train2[["class"]].values.ravel()
neigh2.fit(df_train2[['emb', 'gl2h', 'pad', 'ept', 'is2h', 'imc', 'fpd', 'edad']], 
           train_output2)

# Clasificar los datos de prueba e imprimir los resultados
test_expected_output2 = df_test2[["class"]].values.ravel()
print("\nSalida esperada:", test_expected_output2)
test_automatic_output2 = neigh2.predict(
    df_test2[['emb', 'gl2h', 'pad', 'ept', 'is2h', 'imc', 'fpd', 'edad']])
print("\nSalida obtenida:", test_automatic_output2)

# Contabilizar casos correctos
correct = 0
for i in range(len(test_expected_output2)):
    if test_automatic_output2[i] == test_expected_output2[i]:
        correct += 1

# Calcular la exactitud
accuracy2 = correct / len(test_expected_output2)
print("\nExactitud:", accuracy2)
print("\nExactitud porcentual:", accuracy2 * 100)
print("\nTasa de error:", (1 - accuracy2) * 100)


Salida esperada: [1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0
 1 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0
 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 1 0 1 0 0 1 1 0 1 1 1 1 1 0 0 0 0 1 1 0 0 1
 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 1]

Salida obtenida: [0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0
 1 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0
 0 1 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1
 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1]

Exactitud: 0.7615384615384615

Exactitud porcentual: 76.15384615384615

Tasa de error: 23.84615384615385


Aunque la exactitud es una buena medida de calidad de un clasificador, tiene algunos inconvenientes en problemas reales, debido a que oculta detalles que, en muchos casos, son fundamentales.

* El problema más importante ocurre en datos *no balanceados*: Si una clase tiene muchos más datos que las demás clases, los aciertos en la clase numerosa oculta los errores de las otras clases. Supongamos un problema en el que una clase tiene el 99% de los datos, entonces, mientras no haya errores en esta clase, el error se mantendrá por abajo del 1%. En casos reales, es frecuente tener datos no bien balancedos. Esto ocurre con la mayoría de las enfermedades; incluso la diabetes tiene una frecuencia inferior al 10%, por lo tanto, mientras nuestro sistema no diagnostique como dibético a alguien sano, el error se mantendrá inferior al 10%. La Progeria de Hutchinson–Gilford tiene una incidencia inferior a 1 en 8 millones. Otro caso semejante sería la identificación de terroristas en un aeropuerto o la identificación de fraudes en hipotecas (inferior al 1%). 

* Cuando hay más de dos clases, la exactitud nos da una estimación del error promedio en todas las clases, pero oculta el error en clases específicas.

* Un tercer problemas es el costo del error; este problema surge de la diferencia entre el error cometido al considerar como positivo un caso negativo y el error al calificar como negativo un caso positivo. Clasificar como sano a un paciente enfermo, por ejemplo, suele ser más delicado que clasificar como enfermo a un paciente sano.

## La *Matriz de confusión*

Una matriz de confusión es una matriz de tamaño $N\times N$, siendo $N$ el número de clases. Esta matriz compara la distribución real de elementos por clase contra las clases identificadas por el algoritmo. En esta matriz, cada renglón presenta la cantidad de elementos pertenecientes a una clase (valor real/esperado) mientras que cada columna en el renglón representa la cantidad de elementos en una de las clases (o viceversa.

Consideremos la matriz de confusión para el ejemplo de calificaciones:

In [5]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(test_expected_output, test_automatic_output))

[[4 0 0]
 [0 4 0]
 [0 0 2]]


En este caso: 

* Existen 4 elementos pertenecientes a la clase '0', todos clasificados (correctamente) en la clase '0'.
* Existen 2 elementos pertenecientes a la clase '1', todos clasificados (correctamente) en la clase '1'.
* Existen 4 elementos pertenecientes a la clase '2', todos clasificados (correctamente) en la clase '2'.

Los elementos clasificados correctamente se localizan sobre la diagonal de la matriz de confusión.

Consideremos ahora la matriz de confusión para los datos de diabetes:

In [12]:
print("Datos de prueba:", len(df_test2))
print("Datos de prueba con class=0:", len(df_test2[df_test2["class"]==0]))
print("Datos de prueba con class=1:", len(df_test2[df_test2["class"]==1]))
print("\nExactitud porcentual:", accuracy2 * 100)
print("\nMatriz de confusión\n", 
      confusion_matrix(test_expected_output2, test_automatic_output2))

Datos de prueba: 130
Datos de prueba con class=0: 80
Datos de prueba con class=1: 50

Exactitud porcentual: 76.15384615384615

Matriz de confusión
 [[74  6]
 [25 25]]


En este caso: 

* Existen 80 elementos pertenecientes a la clase '0', de los cuales, sólo 74 fueron clasificados correctamente, mientras que 6 fueron clasificados como pertenecientes a la clase '1'.
* Existen 50 elementos pertenecientes a la clase '1' y sólo 25 de ellos clasificados correctamente.

Nuevamente, los elementos clasificados correctamente se localizan sobre la diagonal de la matriz de confusión. 

Es sobresaliente que mientras que la exactitud global es del 76%, en los casos más importantes (pacientes enfermos), la exactitud es tan solo del 50%.

Otro punto a destacar en este ejemplo es que al haber sólo dos clases, el problema puede replantearse en términos de una sóla clase (diabético), de manera que 'class' = '0' representa un caso negativo (la muestra no pertenece a la clase) y 'class' = 1 constituye un caso positivo (la muestra pertenece a la clase). De esta manera, de los datos de prueba 80 no pertenecen a la clase (son negativos) y 50 si pertenecen a la clase (casos positivos). Entonces, la matriz de confusión puede interpretarse de la siguiente manera:

* 74 resultados son 'verdaderos negativos'.
* 6 resultados son 'falsos positivos' (clasificados erróneamente como positivos).
* 25 resultados son 'verdaderos positivos'.
* 25 resultados son 'falsos negativos' (clasificados erróneamente como negativos).

### Métricas derivadas de la matriz de confusión

La matriz de confusión permite definir una variedad de medidas de calidad en la clasificación (o en la predicción). 

Dada la matriz de confusión:

$$\mathbf{M} = 
\begin{bmatrix}
    M_{11}       & M_{12} & M_{13} & \dots & M_{1n} \\
    M_{21}       & M_{22} & M_{23} & \dots & M_{2n} \\
    \vdots \\
    M_{n1}       & M_{n2} & M_{n3} & \dots & M_{nn}
\end{bmatrix}
$$

Se definen los siguientes conceptos:

* **Verdaderos positivos de la clase $i$**. Es la cantidad de elementos pertenecientes a la clase $i$ que fueron identificados correctamente:<br>
$$Vp_i = M_{i,\ i}$$<br>

* **Falsos positivos de la clase $i$**. Es el número de elementos clasificados en la clase $i$ que en realidad pertenecen a la clase $j\ne i$:<br>
$$Fp_i = \sum_{j\ne i} M_{j,\ i}$$
también se le conoce como *Error tipo I* o *falsa alarma*<br><br>

* **Verdaderos negativos de la clase $i$**. Es el número de elementos ajenos a la clase $i$ que fueron clasificados como negativos, o como pertenecientes a alguna otra clase $j\ne i$, si hay más de dos clases:<br><br>
$$Vn_i = \sum_{k\ne i,\ j\ne i} M_{k,\ j}$$ <br>

* **Falsos negativos de la clase $i$**. Es el número de elementos pertenecientes a la clase $i$ que fueron clasificados, erróneamente, en otra clase $j\ne i$:<br>
$$Fn_i = \sum_{i\ne j} M_{i,\ j}$$
Es también llamado *Error de tipo II* o *desacierto* (*miss*).<br><br>

Y a partir de estos conceptos, se define una variedad de medidas de calidad, entre las que se encuentran las siguientes:

* **Exactitud (*accuracy*)**. Es la proporción de predicciones correctas sobre el total de elementos en la muestra:<br><br>
$$Acc = \frac{\sum Vp_i}{\sum N_i}$$<br>
siendo $N_i$ el número de elementos en la clase $i$. También se le conoce como *veracidad*.<br><br>

* **Valor predictivo positivo de la clase $i$** o **Precisión de la clase $i$**. Es la proporción de verdaderos positivos de una clase con respecto al total de elementos clasificados como pertenecientes a la clase:<br><br>
$$Vpp_i = \frac{Vp_i}{Vp_i + Fp_i}$$<br>

* **Valor predictivo negativo**. Es la proporción de falsos positivos en una clase, con respecto al total de elementos clasificados como no pertenecientes a la clase:<br>
$$Vpn_i = \frac{Fp_i}{Vn_i + Fn_i}$$<br>

* **Sensibilidad (*recall*) de la clase $i$**. Es la proporción de elementos pertenecientes a la clase $i$ que fueron identificados correctamente:<br>
$$Sen_i = \frac{Vp_i}{N_i}$$<br>
También se le suele llamar *tasa positiva verdadera*, *tasa de éxito* o *tasa de detección* y en inglés *recall*.<br><br>

* **Especificidad de la clase $i$** o **Selectividad de la clase $i$**. Es la proporción de elementos no pertenecientes a la clase $i$ que fueron correctamente identificados como no pertenecientes a la clase $i$:<br><br>
$$Esp_i = \frac{Vn_i}{\sum_{j\ne i} N_j}$$
Es también llamada *tasa negativa verdadera*.<br><br>

* **Valor F1 de la clase $i$**. Es una medida compuesta de la exactitud de una prueba derivada a partir de la precisión y la sensibilidad.<br>
$$F1_i = \frac{Vpp_i\times Sen_i}{Vpp_i + Sen_i}$$
También es llamdo simplemente *valor F* o *escore F*.<br><br>

Analicemos algunas de estas medidas para nuestros ejemplos y modificamos los resultados de calificaciones para hacerlo más interesante.

In [9]:
import sklearn.metrics as sm

print("Ejemplo calificaciones", 
      '\nExactitud: ', sm.accuracy_score(test_expected_output, test_automatic_output), 
      '\nValor-F1: ', sm.f1_score(test_expected_output, test_automatic_output, average=None),
      '\n...')

dummy_aut = np.array([2, 0, 1, 0, 2, 1, 0, 1, 1, 0])
print("\nSalida esperada:   ", test_expected_output, 
      "\nSalida modificada: ", dummy_aut, 
      "\n\nMatriz de confusión:\n", 
      confusion_matrix(test_expected_output, dummy_aut), 
      '\n\nExactitud: ', sm.accuracy_score(test_expected_output, dummy_aut), 
      "\n\nReporte:\n", sm.classification_report(test_expected_output, dummy_aut))


Ejemplo calificaciones 
Exactitud:  1.0 
Valor-F1:  [1. 1. 1.] 
...

Salida esperada:    [1 0 0 0 1 1 0 2 2 1] 
Salida modificada:  [2 0 1 0 2 1 0 1 1 0] 

Matriz de confusión:
 [[3 1 0]
 [1 1 2]
 [0 2 0]] 

Exactitud:  0.4 

Reporte:
               precision    recall  f1-score   support

           0       0.75      0.75      0.75         4
           1       0.25      0.25      0.25         4
           2       0.00      0.00      0.00         2

   micro avg       0.40      0.40      0.40        10
   macro avg       0.33      0.33      0.33        10
weighted avg       0.40      0.40      0.40        10



In [15]:
print("Matriz de confusión\n", 
      confusion_matrix(test_expected_output2, test_automatic_output2), "\n")
print("Reporte en el ejemplo diabetes:\n", 
      sm.classification_report(test_expected_output2, test_automatic_output2))

Matriz de confusión
 [[74  6]
 [25 25]] 

Reporte en el ejemplo diabetes:
               precision    recall  f1-score   support

           0       0.75      0.93      0.83        80
           1       0.81      0.50      0.62        50

   micro avg       0.76      0.76      0.76       130
   macro avg       0.78      0.71      0.72       130
weighted avg       0.77      0.76      0.75       130

